In [ ]:
# 导入包(PypI)
import os
import re
import time
import json
import random
import pymongo
import pandas as pd

from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm.notebook import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

##### 启动 Chrome 浏览器实例：

打开 **cmd** 命令提示符<输入以下命令（将 `your Chrome.exe path` 替换为您的 Chrome 浏览器路径）：
```bash
& "your Chrome.exe path" --remote-debugging-port=9222 --user-data-dir="C:\selenum\AutomationProfile"
```

- 请将your Chrome.exe path替换为您的Chrome浏览器所在路径，例如<br>`C:\Program Files\Google\Chrome\Application\chrome.exe`
- 配置 chromedriver 相关信息，请参考官方文档：[ChromeDriver](https://developer.chrome.com/docs/chromedriver)

In [ ]:
# 配置Chrome浏览器
service = Service("C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe")

options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')# 远程调控模式启用
options.add_argument('--incognito')# 隐身/无痕模式启用
browser = webdriver.Chrome(service=service, options=options)
action = ActionChains(browser)

In [ ]:
# 定义全局变量
key_word = ""
num = 0

def selenium_test():
    global key_word, num# 使用全局变量
    browser.get('https://www.xiaohongshu.com/explore')

    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    # 检查是否成功登录小红书
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            time.sleep(10)
            print('暂未登录，请手动登录')
            print('检查时间:',time.ctime())
            continue
        else:
            print('登录成功')
            print('检查时间:',time.ctime())
            time.sleep(3)
            break

    print("即将开始检查网页加载状态...")
    print("如果网页进入人机验证页面，请先手动完成验证。")
    print("请在文本框中根据提示输入搜索关键词和笔记爬取数量。")
    key_word = input("搜索关键词：")
    num = input("笔记爬取数量：")
    num = int(num)

    url = f'https://www.xiaohongshu.com/search_result?keyword={key_word}&source=web_explore_feed'
    browser.get(url)
    time.sleep(3)

    # 检查页面是否加载成功
    while True:
        if key_word in browser.title:
            print('加载成功')
            print('检查时间:',time.ctime())
            break
        else:
            time.sleep(2)

selenium_test()

In [ ]:
# 通过模拟点击更改模式
mode = browser.find_element(By.XPATH, '//*[@id="search-type"]/div/div/div[2]')
mode.click()# 模拟鼠标点击
print('已自动更改模式为图文。')

# 通过模拟点击更改排序方式
sort_order = ['综合','最新','最热']
print("请在文本框中根据提示输入对应数字来选择排序方式。")
for i, order in enumerate(sort_order, 1):
    print(f'{i}.{order}')
selected_order = input("请选择排序方式:")

try:
    selected_order_index = int(selected_order)
    selected_order_text = sort_order[selected_order_index-1]
    element = browser.find_element(By.XPATH, '//*[@id="global"]/div[2]/div[2]/div/div[1]/div[2]')
    action.move_to_element(element).perform()# 模拟鼠标悬停
    menu = browser.find_element(By.CLASS_NAME, 'dropdown-items')
    option = menu.find_element(By.XPATH, f'/html/body/div[4]/div/li[{selected_order_index}]')
    option.click()# 模拟鼠标点击
    print('已选择排序方式为:',selected_order_text)
    print('检查时间:',time.ctime())
except Exception as e:
    print(e)

In [ ]:
authorName_list = []
likeNr_list = []
URL_list = []
userURL_list = []

# 进度条: 实时跟踪已爬取的笔记数量
qbar = tqdm(total=num)

# 定义函数：解析网页内容，爬取数据
def parsePage(key_word):
    response = TextResponse(url=browser.current_url, body=key_word, encoding='utf-8')# 将key_word转换为Response对象
    selector = Selector(response)
    divs = selector.xpath('//div[contains(@class, "feeds-container")]/section/div')# 选中网页中包含笔记信息的部分

    # 定义循环：遍历divs获取每一篇笔记的信息
    for div in divs:
        # 选择并提取网页数据
        try:
            author_name = div.xpath('./div/div/a/span/text()').extract_first()# 作者名字
            like_nr = div.xpath('string(./div/div/span)').extract_first()# 获赞数量
            url = div.xpath('./div/a/@href').extract_first().split('/')[-1]# 笔记URL
            user_url = div.xpath('./div/div/a/@href').extract_first()# 用户URL

            authorName_list.append(author_name)
            likeNr_list.append(like_nr)
            URL_list.append(url)
            userURL_list.append(user_url)

            time.sleep(0.35)

            qbar.update(min(1, num - len(URL_list)))
            
        except:
            pass

# 定义循环：检查是否已经爬取足够数量的笔记，或是否已经达到页面底部
while len(URL_list) < num:
    if '- THE END -' in browser.page_source:
        print(f"当前与{key_word}有关的笔记数量少于 {num}")
        print('检查时间:',time.ctime())
        break
    
    parsePage(browser.page_source)# 解析当前页面
    time.sleep(0.3)
    browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')# 模拟鼠标滚动
    wait_time=random.uniform(3,5)# 生成随机等待时间
    time.sleep(wait_time)

if len(URL_list) > num:
    URL_list = URL_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    userURL_list = userURL_list[:num]

qbar.close()

In [ ]:
if (len(authorName_list) == len(likeNr_list) == len(URL_list) == len(userURL_list) == num):
    print("所有属性列表长度均为", num,"，爬取成功！")
    print('检查时间:',time.ctime())
else:
    min_length = min(len(authorName_list), len(likeNr_list), len(URL_list), len(userURL_list))
    print("当前属性列表长度最小值为", min_length, "请重新运行上一代码单元，直至所有属性列表长度均为", num, "！")
    print('检查时间:',time.ctime())

In [ ]:
# 数据清洗
likeNr_list_cleaned = []
for like_nr in likeNr_list:
    if 'w' in like_nr:
        like_nr = like_nr.replace('w', '')
        like_nr = float(like_nr) * 10000
        like_nr = int(like_nr)
    else:
        like_nr = int(like_nr)
    
    likeNr_list_cleaned.append(like_nr)

userURL_list_cleaned = [url.split('/')[-1] for url in userURL_list]

In [ ]:
print("以下为清洁数据示例：")
for i in range(3):
    print("author_name:", authorName_list[i])
    print("like_nr:", likeNr_list_cleaned[i])
    print("url:", URL_list[i])
    print("user_url:", userURL_list_cleaned[i])
    print("------")

In [ ]:
commentNr_list = []
content_list = []
datePublished_list = []
images_list = []
starNr_list = []

# 进度条: 实时跟踪已爬取的笔记数量
qbar = tqdm(total=len(URL_list))

# 定义循环：遍历items中的每一篇笔记
for url in URL_list:
    whole_url = 'https://www.xiaohongshu.com/discovery/item/' + url# 构造完整笔记URL
    browser.get(whole_url)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))# 等待笔记页面加载完成
    html = browser.page_source

    selector = Selector(text = html)# 解析网页内容
    
    try:
        # 选择并提取网页数据
        comment_nr = selector.xpath('//*[@class="total"]/text()').extract_first()# 评论数量
        content = selector.xpath('//*[@name="description"]/@content').extract_first()# 内容
        datePublished = selector.xpath('//*[@class="date"]/text()').extract_first()# 发布时间
        images = selector.xpath('//*[@name="og:image"]/@content').extract_first()# 图片
        images = images + '.jpg' if images else None# 追加图片链接
        star_nr = selector.xpath('//*[@class="count"]/text()').extract_first()# 收藏数量

        commentNr_list.append(comment_nr)
        content_list.append(content)
        datePublished_list.append(datePublished)
        images_list.append(images)
        starNr_list.append(star_nr)

    except:
        continue
    
    qbar.update(1)
    
    wait_time = random.uniform(0.5, 2)
    time.sleep(wait_time)

qbar.close()

In [ ]:
if (len(commentNr_list) == len(content_list) == len(datePublished_list) == len(images_list) == len(starNr_list) == num):
    print("所有属性列表长度均为", num,"，爬取成功！")
    print('检查时间:',time.ctime())
else:
    min_length = min(len(commentNr_list), len(content_list), len(datePublished_list), len(images_list), len(starNr_list))
    print("当前属性列表长度最小值为", min_length, "请重新运行上一代码单元，直至所有属性列表长度均为", num, "！")
    print('检查时间:',time.ctime())

In [ ]:
# 数据清洗
# 定义函数：使用正则表达式提取所有数字
def extract_number(x):
    # 检查变量类型
    if x is None or not isinstance(x, str):
        return None
    numbers = re.findall(r'\d+', x)
    if numbers:
        return int(numbers[0])# 转换数据
    else:
        return None

commentNr_list = [extract_number(commentNr) for commentNr in commentNr_list]
starNr_list = [extract_number(star_nr) for star_nr in starNr_list]

# 定义函数：使用正则表达式提取所有日期
def extract_date(datePublished):
    if datePublished is None or not isinstance(datePublished, str):
        return None
    else:
        match = re.search(r'\d{4}-\d{2}-\d{2}', datePublished)
        if match:
            return match.group()# 转换数据
        else:
            return None

datePublished_list = [datetime.strptime(extract_date(date), "%Y-%m-%d") if extract_date(date) else None for date in datePublished_list]

In [ ]:
print("以下为清洁数据示例：")
for i in range(3):
    print("comment_nr:", commentNr_list[i])
    print("content:", content_list[i])
    print("datePublished:", datePublished_list[i])
    print("images:", images_list[i])
    print("star_nr:", starNr_list[i])
    print("------")

In [ ]:
authorCollectNr_list = []
authorFansNr_list = []
authorNoteNr_list = []

wait_time = random.uniform(1, 2)

# 进度条: 实时跟踪已爬取的笔记数量
qbar = tqdm(total=len(userURL_list))

# 定义循环：遍历items中的每一篇笔记
for user_url in userURL_list_cleaned:
    whole_url = 'https://www.xiaohongshu.com/user/profile/' + user_url# 构造完整个人页面URL
    browser.get(whole_url)

    while True:
        previous_page_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")# 模拟鼠标滚动到页面底部
        time.sleep(wait_time)

        current_page_height = browser.execute_script("return document.body.scrollHeight")
        # 检查滚动后页面高度变化
        if current_page_height == previous_page_height:
            break
        previous_page_height = current_page_height

    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="userPostedFeeds"]//section')))# 等待笔记页面加载完成
    # 解析完成加载的网页内容
    html = browser.page_source
    selector = Selector(text=html)
    
    # 选择并提取网页数据
    author_collect_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[3]/span[@class="count"]/text()').extract_first()# 作者获赞与收藏数量
    author_fans_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[2]/span[@class="count"]/text()').extract_first()# 作者粉丝数量
    author_note_nr = len(selector.xpath('//*[@id="userPostedFeeds"]//section'))# 作者笔记数量

    authorCollectNr_list.append(author_collect_nr)
    authorFansNr_list.append(author_fans_nr)
    authorNoteNr_list.append(author_note_nr)
    
    qbar.update(1)
    time.sleep(wait_time)

qbar.close()

In [ ]:
if (len(authorCollectNr_list) == len(authorFansNr_list) == len(authorNoteNr_list) == num):
    print("所有属性列表长度均为", num,"，爬取成功！")
    print('检查时间:',time.ctime())
else:
    min_length = min(len(authorCollectNr_list), len(authorFansNr_list), len(authorNoteNr_list))
    print("当前属性列表长度最小值为", min_length, "请重新运行上一代码单元，直至所有属性列表长度均为", num, "！")
    print('检查时间:',time.ctime())

In [ ]:
# 数据清洗
# 定义函数：使用正则表达式提取所有数字
def extract_largeNumber(element):
    match = re.search(r'(\d+(\.\d+)?万?)', element)
    if match:
        number_str = match.group(1)
        if '万' in number_str:
                number = float(number_str.replace('万', ''))
                number = int(number * 10000)
        else:
            number = int(number_str)
        return number
    else:
        return None

authorCollectNr_list_cleaned = [extract_largeNumber(author_collect_nr) for author_collect_nr in authorCollectNr_list]
authorFansNr_list_cleaned = [extract_largeNumber(author_fans_nr) for author_fans_nr in authorFansNr_list]

In [ ]:
print("以下为清洁数据示例：")
for i in range(3):
    print("author_collect_nr:", authorCollectNr_list_cleaned[i])
    print("author_fans_nr:", authorFansNr_list_cleaned[i])
    print("author_note_nr:", authorNoteNr_list[i])
    print("------")

In [ ]:
col_name = f"{key_word}{selected_order_text}笔记Top{num}"

# 连接MongoDB
myclient=pymongo.MongoClient("mongodb://localhost:27017/")
mydb=myclient["小红书关键词笔记数据"]#进行操作的库名
mycol=mydb[col_name]#进行操作的集合名

In [ ]:
dic={
    "author_collect_nr": authorCollectNr_list_cleaned,# 作者获赞与收藏数量
    "author_fans_nr": authorFansNr_list_cleaned,# 粉丝数量
    "author_name": authorName_list,# 作者名字
    "author_note_nr": authorNoteNr_list,# 作者笔记数量
    "comment_nr": commentNr_list,# 笔记评论数量
    "content": content_list,# 笔记内容
    "datePublished": datePublished_list,# 笔记发布日期
    "images": images_list,# 笔记封面图片
    "like_nr": likeNr_list_cleaned,# 笔记获赞数量
    "url": URL_list,# 笔记URL
    "user_url": userURL_list_cleaned# 作者URL
    }

df = pd.DataFrame.from_dict(dic)
df = df[~df.duplicated(keep='first')]# 检索并删除所有属性值都相同的行,即保留第一次出现的行，删除后续的重复行
print("删除", num-len(df), "行重复行后剩余", len(df), "行。")
print('检查时间:',time.ctime())
df.head(10)

In [ ]:
mycol.insert_many(df.to_dict('records'))

In [ ]:
myclient.close()# 关闭MongoDB客户端连接